# Python -> LLVM compiler

In [2]:
%run ~/.jupyter/config.ipy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import ast_to_llvm.python_to_core as ptc
import ast_to_llvm.core_to_llvm as ctl

from ctypes import CFUNCTYPE, c_double, c_int

# tests

In [23]:
func = """
def f(a, b):
    c = a + b
    return c
"""

In [24]:
core = ptc.PythonVisitor()(func)[0]


def f(a, b):
    c = a + b
    return c

PYTHON
('Module',
 {'body': [('FunctionDef',
            {'args': ('arguments',
                      {'args': [('arg', {'annotation': None, 'arg': "'a'"}),
                                ('arg', {'annotation': None, 'arg': "'b'"})],
                       'defaults': [],
                       'kw_defaults': [],
                       'kwarg': None,
                       'kwonlyargs': [],
                       'vararg': None}),
             'body': [('Assign',
                       {'targets': [('Name',
                                     {'ctx': ('Store', {}), 'id': "'c'"})],
                        'value': ('BinOp',
                                  {'left': ('Name',
                                            {'ctx': ('Load', {}), 'id': "'a'"}),
                                   'op': ('Add', {}),
                                   'right': ('Name',
                                             {'ctx': ('Load', {}),
                  

In [28]:
f = ctl.LLVMEmitter().visit(core)

CORE
('Func',
 {'args': [('Var', {'ref': "'a'", 'type': None}),
           ('Var', {'ref': "'b'", 'type': None})],
  'body': [('Assign',
            {'ref': "'c'",
             'type': None,
             'val': ('PrimOp',
                     {'args': [('Var', {'ref': "'a'", 'type': None}),
                               ('Var', {'ref': "'b'", 'type': None})],
                      'fn': "'add#'"})}),
           ('Return', {'val': ('Var', {'ref': "'c'", 'type': None})})],
  'fname': "'f'"})

LLVM
define i32 @"fname"(i32 %".1", i32 %".2") 
{
entry:
  %"a" = alloca i32
  store i32 %".1", i32* %"a"
  %"b" = alloca i32
  store i32 %".2", i32* %"b"
  %".6" = load i32, i32* %"a"
  %".7" = load i32, i32* %"b"
  %".8" = add i32 %".6", %".7"
  %"c" = alloca i32
  store i32 %".8", i32* %"c"
  %".10" = load i32, i32* %"c"
  ret i32 %".10"
}



In [26]:
ctl.llvm_init()
engine = ctl.create_llvm_engine()
ctl.compile_code(engine, str(f))

In [27]:
func_ptr = engine.get_function_address("fname")
cfunc = CFUNCTYPE(c_int, c_int, c_int)(func_ptr)
res = cfunc(2, 3)
print("fname(...) =", res)

fname(...) = 5
